# Module 12: Introduction to AI Agents

## What Are AI Agents?

AI agents are **autonomous systems** that use Large Language Models (LLMs) as their reasoning engine to **decide which actions to take** in order to achieve a goal. Unlike simple LLM calls where you send a prompt and get a response, agents can:

- **Use tools** to interact with the outside world (search the web, run code, query databases)
- **Make decisions** about which tool to use and when
- **Iterate** on their approach based on intermediate results
- **Chain multiple steps** together to solve complex problems

### Simple LLM Call vs. Agent

| Simple LLM Call | AI Agent |
|---|---|
| Single input, single output | Multi-step reasoning |
| No external tools | Can call tools (APIs, databases, code) |
| Stateless | Maintains context across steps |
| Deterministic flow | Dynamic flow based on LLM decisions |
| Limited to training data | Can fetch real-time information |

### Historical Note

This repository contains `langchain_agent.ipynb` which demonstrates an earlier approach using deprecated LangChain APIs (`LLMChain`, `Tool.from_function`, `initialize_agent`). In this module, we use the **modern LangChain APIs** (`@tool` decorator, `create_react_agent`, `AgentExecutor`) which are more robust and better supported.

---

## 1. Setup

First, let's install the required packages and configure our environment.

In [ ]:
!pip install -q openai python-dotenv langchain langchain-openai langchain-community requests

In [ ]:
import os
import json
import random
from datetime import datetime

from dotenv import load_dotenv
import requests
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_react_agent, create_tool_calling_agent, AgentExecutor
from langchain import hub

# Load environment variables
load_dotenv("/home/amir/source/.env")

print("Environment loaded successfully.")

In [ ]:
# Initialize OpenAI client (for the raw API demo)
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize LangChain LLM (for agent demos)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("Clients initialized.")

---

## 2. The Agent Loop

At the heart of every AI agent is a simple loop:

```
                +------------------+
                |                  |
                v                  |
  +----------+     +----------+   |
  | OBSERVE  |---->|  THINK   |   |
  +----------+     +----------+   |
       ^                |         |
       |                v         |
  +----------+     +----------+   |
  | OBSERVE  |<----+   ACT    +---+
  | (result) |     +----------+
  +----------+
       |
       v
  +----------+
  |  FINAL   |
  | ANSWER   |
  +----------+
```

1. **Observe**: The agent receives input (user query or tool result)
2. **Think**: The LLM reasons about what to do next
3. **Act**: The agent calls a tool or returns a final answer
4. **Observe** (again): If a tool was called, the agent observes the result and loops back to Think

This cycle continues until the agent decides it has enough information to provide a final answer.

### Demo: Manual Agent Loop with OpenAI Function Calling

Let's implement this loop manually using the OpenAI API to understand what's happening under the hood.

In [ ]:
# Define a simple calculator tool for OpenAI function calling
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate a mathematical expression. Supports basic arithmetic (+, -, *, /, **) and common math functions.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 3 * 4'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

def calculator(expression: str) -> str:
    """Safely evaluate a mathematical expression."""
    try:
        # Only allow safe math operations
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return f"Error: Expression contains invalid characters."
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {e}"

print("Tool defined. Testing calculator:")
print(f"  calculator('2 + 3 * 4') = {calculator('2 + 3 * 4')}")
print(f"  calculator('(10 + 5) / 3') = {calculator('(10 + 5) / 3')}")

In [ ]:
def run_agent_loop(user_message: str) -> str:
    """
    Manually implement the Observe -> Think -> Act -> Observe loop
    using OpenAI's function calling API.
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the calculator tool when you need to perform math calculations. Always show your reasoning."},
        {"role": "user", "content": user_message}
    ]

    print(f"User: {user_message}")
    print("=" * 60)

    max_iterations = 5
    for i in range(max_iterations):
        print(f"\n--- Iteration {i + 1} ---")

        # THINK: Send messages to the LLM
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools_schema,
            tool_choice="auto"
        )

        assistant_message = response.choices[0].message
        messages.append(assistant_message)

        # Check if the LLM wants to call a tool (ACT)
        if assistant_message.tool_calls:
            for tool_call in assistant_message.tool_calls:
                func_name = tool_call.function.name
                func_args = json.loads(tool_call.function.arguments)
                print(f"  [ACT] Calling tool: {func_name}({func_args})")

                # Execute the tool
                if func_name == "calculator":
                    result = calculator(func_args["expression"])
                else:
                    result = f"Unknown tool: {func_name}"

                print(f"  [OBSERVE] Tool result: {result}")

                # Feed the result back to the LLM
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })
        else:
            # No tool call means the agent is done (FINAL ANSWER)
            final_answer = assistant_message.content
            print(f"\n  [FINAL ANSWER] {final_answer}")
            return final_answer

    return "Max iterations reached without a final answer."


# Test the agent loop
run_agent_loop("What is 47 * 83 + 129?")

In [ ]:
# A more complex example requiring multiple tool calls
run_agent_loop("I bought 15 items at $23.50 each. After a 12% discount, what's my total? Also, what's 20% tip on that total?")

**Key takeaway:** The agent loop is straightforward:
1. The LLM receives the conversation so far and decides whether to call a tool or give a final answer.
2. If it calls a tool, we execute it and feed the result back.
3. We repeat until we get a final answer.

Frameworks like LangChain automate this loop so you don't have to write it manually every time.

---

## 3. Building Tools with LangChain

The modern way to define tools in LangChain is with the `@tool` decorator. This automatically:
- Extracts the tool name from the function name
- Extracts the description from the docstring
- Infers the input schema from type hints

Let's build four useful tools.

In [ ]:
@tool
def calculator_tool(expression: str) -> str:
    """Evaluate a mathematical expression. Supports basic arithmetic like
    addition (+), subtraction (-), multiplication (*), division (/), and
    exponentiation (**). Example: '2 + 3 * 4' returns '14'."""
    try:
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Expression contains invalid characters."
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


@tool
def get_current_datetime() -> str:
    """Get the current date and time. Returns the date in YYYY-MM-DD HH:MM:SS
    format along with the day of the week. Useful when the user asks about
    today's date or current time."""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S (%A)")


@tool
def word_counter(text: str) -> str:
    """Count the number of words, characters, and sentences in the given text.
    Returns a summary with word count, character count, and sentence count."""
    words = len(text.split())
    characters = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    return f"Words: {words}, Characters: {characters}, Sentences: {sentences}"


@tool
def web_fetcher(url: str) -> str:
    """Fetch the text content from a given URL. Returns the first 2000
    characters of the page text. Useful for retrieving information from
    web pages."""
    try:
        response = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        # Simple text extraction: strip HTML tags
        from html.parser import HTMLParser
        class TextExtractor(HTMLParser):
            def __init__(self):
                super().__init__()
                self.texts = []
                self._skip = False
            def handle_starttag(self, tag, attrs):
                if tag in ('script', 'style'):
                    self._skip = True
            def handle_endtag(self, tag):
                if tag in ('script', 'style'):
                    self._skip = False
            def handle_data(self, data):
                if not self._skip:
                    self.texts.append(data.strip())
        extractor = TextExtractor()
        extractor.feed(response.text)
        text = ' '.join(t for t in extractor.texts if t)
        return text[:2000]
    except Exception as e:
        return f"Error fetching URL: {e}"


# Collect all tools
tools = [calculator_tool, get_current_datetime, word_counter, web_fetcher]

print("Tools created successfully!")
print()

In [ ]:
# Inspect the tool schemas that LangChain auto-generated
for t in tools:
    print(f"Tool: {t.name}")
    print(f"  Description: {t.description}")
    print(f"  Schema: {t.args_schema.model_json_schema()}")
    print()

---

## 4. Agent Types

LangChain supports several agent types. The two most important are:

### ReAct Agent (`create_react_agent`)
- Based on the **ReAct** paper (Yao et al., 2023)
- Interleaves **Re**asoning and **Act**ing in a single prompt
- The LLM generates explicit `Thought:`, `Action:`, and `Observation:` traces
- Works with any LLM (including those without native function calling)
- Best for: transparency, debugging, older models

### Tool-Calling Agent (`create_tool_calling_agent`)
- Uses the LLM's **native function/tool calling** capability (e.g., OpenAI's function calling)
- The LLM directly outputs structured tool calls in its response
- More efficient since the model was trained for this
- Best for: production use with modern LLMs (GPT-4, Claude, Gemini)

Let's build both using the same set of tools and compare their behavior.

### 4a. Tool-Calling Agent

In [ ]:
# Build a tool-calling agent using modern LangChain APIs
tool_calling_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use your tools to answer questions. Always explain your reasoning."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

tool_calling_agent = create_tool_calling_agent(llm, tools, tool_calling_prompt)
tool_calling_executor = AgentExecutor(
    agent=tool_calling_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("Tool-calling agent ready.")

In [ ]:
# Test the tool-calling agent
result = tool_calling_executor.invoke(
    {"input": "What's today's date, and how many days are left until New Year's Eve 2025?"}
)
print("\nFinal Answer:", result["output"])

### 4b. ReAct Agent

In [ ]:
# Build a ReAct agent
# The ReAct agent needs a specific prompt format with {tools}, {tool_names}, {input}, and {agent_scratchpad}
react_prompt = hub.pull("hwchase17/react")

react_agent = create_react_agent(llm, tools, react_prompt)
react_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("ReAct agent ready.")

In [ ]:
# Test the ReAct agent with the same question
result = react_executor.invoke(
    {"input": "What's today's date, and how many days are left until New Year's Eve 2025?"}
)
print("\nFinal Answer:", result["output"])

**Comparison Notes:**

| Feature | Tool-Calling Agent | ReAct Agent |
|---|---|---|
| Reasoning visibility | Implicit (in model's internal reasoning) | Explicit (`Thought:` traces) |
| Efficiency | More efficient (native API feature) | Slightly more tokens (text-based reasoning) |
| Model compatibility | Requires models with tool-calling support | Works with any LLM |
| Debugging | Harder to debug | Easy to follow reasoning chain |
| Prompt format | Flexible | Requires specific `{tools}`, `{tool_names}`, `{agent_scratchpad}` |

---

## Exercise 1: Build a Weather Agent

Create a mock weather tool and an agent that uses it. The tool should return hardcoded weather data for demonstration purposes.

**Tasks:**
1. Create a `get_weather` tool using the `@tool` decorator
2. Build a tool-calling agent with both the weather tool and the calculator tool
3. Ask the agent: "What's the weather in Tokyo and New York? What's the temperature difference?"

In [ ]:
# Exercise 1: YOUR CODE HERE

# Step 1: Create a mock weather tool
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city. Returns temperature,
    humidity, and conditions."""
    # TODO: Create a dictionary of mock weather data for a few cities
    # Return weather info for the requested city, or "City not found" for unknown cities
    weather_data = None  # Replace with a dict of city -> weather info
    pass


# Step 2: Create the agent
weather_tools = None  # Replace with list of tools
weather_agent = None   # Replace with create_tool_calling_agent(...)
weather_executor = None  # Replace with AgentExecutor(...)


# Step 3: Test the agent
# result = weather_executor.invoke({"input": "What's the weather in Tokyo and New York? What's the temperature difference?"})
# print(result["output"])

### Solution

In [ ]:
# Exercise 1: SOLUTION

# Step 1: Create a mock weather tool
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city. Returns temperature in
    Fahrenheit, humidity percentage, and weather conditions."""
    weather_data = {
        "tokyo": {"temp_f": 72, "humidity": 65, "conditions": "Partly Cloudy"},
        "new york": {"temp_f": 58, "humidity": 45, "conditions": "Sunny"},
        "london": {"temp_f": 52, "humidity": 80, "conditions": "Rainy"},
        "paris": {"temp_f": 55, "humidity": 70, "conditions": "Overcast"},
        "sydney": {"temp_f": 85, "humidity": 55, "conditions": "Clear"},
    }
    city_lower = city.lower().strip()
    if city_lower in weather_data:
        w = weather_data[city_lower]
        return f"Weather in {city}: {w['temp_f']}F, Humidity: {w['humidity']}%, Conditions: {w['conditions']}"
    return f"Weather data not available for '{city}'. Available cities: {', '.join(weather_data.keys())}"


# Step 2: Create the agent
weather_tools = [get_weather, calculator_tool]

weather_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful weather assistant. Use your tools to look up weather and perform calculations."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

weather_agent = create_tool_calling_agent(llm, weather_tools, weather_prompt)
weather_executor = AgentExecutor(
    agent=weather_agent,
    tools=weather_tools,
    verbose=True,
    handle_parsing_errors=True
)

# Step 3: Test the agent
result = weather_executor.invoke(
    {"input": "What's the weather in Tokyo and New York? What's the temperature difference?"}
)
print("\nFinal Answer:", result["output"])

---

## 5. ReAct Pattern Deep Dive

The ReAct (Reasoning + Acting) pattern is one of the most important concepts in AI agents. Let's examine how it works in detail.

### The Thought-Action-Observation Trace

In a ReAct agent, the LLM generates a structured trace:

```
Thought: I need to find out what today's date is first.
Action: get_current_datetime
Action Input: {}
Observation: 2025-01-15 14:30:00 (Wednesday)

Thought: Now I know the date. I need to calculate the number of days...
Action: calculator_tool
Action Input: {"expression": "365 - 15"}
Observation: 350

Thought: I now have all the information I need.
Final Answer: Today is January 15, 2025. There are 350 days left...
```

This trace is valuable because:
1. You can see **exactly why** the agent made each decision
2. You can **debug** issues by examining the reasoning chain
3. You can **audit** agent behavior for safety and correctness

Let's walk through a multi-step example with verbose output.

In [ ]:
# Multi-step ReAct example with verbose output
# The agent needs to: get the date, do a calculation, and count words
result = react_executor.invoke(
    {"input": (
        "Please do the following:\n"
        "1. Tell me what today's date is\n"
        "2. Calculate 2 to the power of 16\n"
        "3. Count the words in this sentence: 'The quick brown fox jumps over the lazy dog'"
    )}
)

print("\n" + "=" * 60)
print("Final Answer:", result["output"])

In [ ]:
# You can also access the intermediate steps if you set return_intermediate_steps=True
react_executor_with_steps = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=False,  # Disable verbose printing
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

result = react_executor_with_steps.invoke(
    {"input": "What is 123 * 456? Then count the words in the result."}
)

print("Final Answer:", result["output"])
print("\nIntermediate Steps:")
for i, (action, observation) in enumerate(result["intermediate_steps"]):
    print(f"  Step {i + 1}:")
    print(f"    Tool: {action.tool}")
    print(f"    Input: {action.tool_input}")
    print(f"    Output: {observation}")

---

## 6. Error Handling

In production, tools will fail. Networks go down, APIs return errors, inputs are malformed. A robust agent needs to handle these gracefully.

### Strategies for Error Handling

1. **Tool-level error handling**: Catch exceptions inside the tool and return an error message
2. **Agent-level retry**: The `AgentExecutor` can be configured to handle parsing errors
3. **Fallback behavior**: Provide alternative actions when a tool fails
4. **Graceful degradation**: Return partial results instead of complete failure

In [ ]:
# Example 1: Tool-level error handling
@tool
def division_tool(numerator: float, denominator: float) -> str:
    """Divide the numerator by the denominator. Returns the result of the division."""
    try:
        if denominator == 0:
            return "Error: Division by zero is not allowed. Please provide a non-zero denominator."
        result = numerator / denominator
        return f"{numerator} / {denominator} = {result}"
    except Exception as e:
        return f"Error performing division: {str(e)}"


# Example 2: Tool with timeout and retry logic
@tool
def reliable_web_fetcher(url: str) -> str:
    """Fetch content from a URL with automatic retry on failure. Tries up
    to 3 times before giving up. Returns the first 1500 characters of text."""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(
                url,
                timeout=5,
                headers={"User-Agent": "Mozilla/5.0"}
            )
            response.raise_for_status()
            # Simple text extraction
            from html.parser import HTMLParser
            class TextExtractor(HTMLParser):
                def __init__(self):
                    super().__init__()
                    self.texts = []
                    self._skip = False
                def handle_starttag(self, tag, attrs):
                    if tag in ('script', 'style'):
                        self._skip = True
                def handle_endtag(self, tag):
                    if tag in ('script', 'style'):
                        self._skip = False
                def handle_data(self, data):
                    if not self._skip:
                        self.texts.append(data.strip())
            extractor = TextExtractor()
            extractor.feed(response.text)
            text = ' '.join(t for t in extractor.texts if t)
            return text[:1500]
        except requests.exceptions.Timeout:
            if attempt < max_retries - 1:
                continue
            return f"Error: Request timed out after {max_retries} attempts for URL: {url}"
        except requests.exceptions.HTTPError as e:
            return f"Error: HTTP {e.response.status_code} for URL: {url}"
        except Exception as e:
            if attempt < max_retries - 1:
                continue
            return f"Error: Failed to fetch {url} after {max_retries} attempts. Last error: {str(e)}"


print("Error-handling tools created.")

In [ ]:
# Test error handling: the agent should gracefully handle division by zero
error_handling_tools = [division_tool, calculator_tool]

error_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful math assistant. If a tool returns an error, "
        "explain the error to the user and suggest alternatives."
    )),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

error_agent = create_tool_calling_agent(llm, error_handling_tools, error_prompt)
error_executor = AgentExecutor(
    agent=error_agent,
    tools=error_handling_tools,
    verbose=True,
    handle_parsing_errors=True
)

# This should trigger the division by zero error and the agent should handle it gracefully
result = error_executor.invoke(
    {"input": "What is 100 divided by 0?"}
)
print("\nFinal Answer:", result["output"])

In [ ]:
# Test with an invalid URL to see retry + graceful degradation
error_tools_2 = [reliable_web_fetcher, word_counter]

error_prompt_2 = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful research assistant. If fetching a URL fails, "
        "inform the user and suggest alternatives."
    )),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

error_agent_2 = create_tool_calling_agent(llm, error_tools_2, error_prompt_2)
error_executor_2 = AgentExecutor(
    agent=error_agent_2,
    tools=error_tools_2,
    verbose=True,
    handle_parsing_errors=True
)

result = error_executor_2.invoke(
    {"input": "Please fetch the content from https://thisurldoesnotexist12345.com and count its words."}
)
print("\nFinal Answer:", result["output"])

---

## Exercise 2: Research Assistant Agent

Build a "research assistant" agent that can search for information and summarize it.

**Tasks:**
1. Create a `mock_web_search` tool that returns hardcoded search results
2. Create a `summarize_text` tool that uses the LLM to summarize text
3. Build an agent and test it with: "Search for information about the transformer architecture and summarize the key points"

In [ ]:
# Exercise 2: YOUR CODE HERE

# Step 1: Create a mock web search tool
@tool
def mock_web_search(query: str) -> str:
    """Search the web for information about a given query. Returns relevant
    search results as text."""
    # TODO: Create a dictionary mapping keywords to mock search results
    # Check if any keyword appears in the query and return the corresponding result
    search_results = None  # Replace with dict of keyword -> search result text
    pass


# Step 2: Create a summarization tool
@tool
def summarize_text(text: str) -> str:
    """Summarize the given text into 3-5 key bullet points."""
    # TODO: Use the OpenAI client to summarize the text
    pass


# Step 3: Build the agent
research_tools = None  # Replace with list of tools
research_agent = None   # Replace with create_tool_calling_agent(...)
research_executor = None  # Replace with AgentExecutor(...)

# Step 4: Test the agent
# result = research_executor.invoke(
#     {"input": "Search for information about the transformer architecture and summarize the key points"}
# )
# print(result["output"])

### Solution

In [ ]:
# Exercise 2: SOLUTION

# Step 1: Create a mock web search tool
@tool
def mock_web_search(query: str) -> str:
    """Search the web for information about a given query. Returns relevant
    search results as text."""
    search_db = {
        "transformer": (
            "The Transformer architecture was introduced in the 2017 paper 'Attention Is All You Need' "
            "by Vaswani et al. Key innovations include: (1) Self-attention mechanism that allows the model "
            "to weigh the importance of different parts of the input sequence. (2) Multi-head attention "
            "that allows the model to attend to information from different representation subspaces. "
            "(3) Positional encoding to inject sequence order information since the model has no recurrence. "
            "(4) Encoder-decoder architecture where the encoder processes input and the decoder generates output. "
            "(5) Layer normalization and residual connections for stable training. "
            "The Transformer has become the foundation for models like BERT, GPT, T5, and virtually all "
            "modern large language models. It replaced RNNs and LSTMs as the dominant architecture for NLP tasks."
        ),
        "attention": (
            "Attention mechanisms allow neural networks to focus on relevant parts of the input when "
            "producing output. Scaled dot-product attention computes: Attention(Q,K,V) = softmax(QK^T / sqrt(d_k))V. "
            "Multi-head attention runs multiple attention functions in parallel."
        ),
        "bert": (
            "BERT (Bidirectional Encoder Representations from Transformers) was introduced by Google in 2018. "
            "It uses masked language modeling and next sentence prediction for pre-training."
        ),
    }
    query_lower = query.lower()
    results = []
    for keyword, content in search_db.items():
        if keyword in query_lower:
            results.append(content)
    if results:
        return " ".join(results)
    return f"No results found for query: '{query}'. Try different search terms."


# Step 2: Create a summarization tool
@tool
def summarize_text(text: str) -> str:
    """Summarize the given text into 3-5 concise key bullet points."""
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Summarize the following text into 3-5 concise bullet points. Each bullet should capture a key idea."},
            {"role": "user", "content": text}
        ],
        temperature=0
    )
    return response.choices[0].message.content


# Step 3: Build the agent
research_tools = [mock_web_search, summarize_text]

research_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a research assistant. Use the search tool to find information, "
        "then use the summarization tool to create concise summaries. "
        "Always search first, then summarize the results."
    )),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

research_agent = create_tool_calling_agent(llm, research_tools, research_prompt)
research_executor = AgentExecutor(
    agent=research_agent,
    tools=research_tools,
    verbose=True,
    handle_parsing_errors=True
)

# Step 4: Test the agent
result = research_executor.invoke(
    {"input": "Search for information about the transformer architecture and summarize the key points"}
)
print("\nFinal Answer:", result["output"])

---

## 7. Old vs. New API Comparison

If you look at `langchain_agent.ipynb` in this repository, you'll see the **deprecated** LangChain patterns. Here's a side-by-side comparison showing how to migrate.

### Deprecated Pattern (pre-LangChain 1.0)

```python
# OLD WAY - DEPRECATED
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.prompts import PromptTemplate

# 1. Create LLM
llm = OpenAI(temperature=0)  # Deprecated class

# 2. Define tools using Tool.from_function
my_tool = Tool.from_function(
    func=my_function,
    name="My Tool",
    description="Does something useful"
)

# 3. Create an LLMChain (deprecated)
chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("...")
)

# 4. Initialize agent (deprecated)
agent = initialize_agent(
    tools=[my_tool],
    llm=llm,
    verbose=True
)

# 5. Run
agent.run("Do something")  # .run() is deprecated
```

### Modern Pattern (LangChain 0.2+)

```python
# NEW WAY - RECOMMENDED
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor

# 1. Create LLM (use langchain_openai package)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 2. Define tools using @tool decorator
@tool
def my_tool(input_param: str) -> str:
    """Does something useful."""  # Docstring becomes the description
    return do_something(input_param)

# 3. No more LLMChain needed! Use prompt | llm or agents

# 4. Create agent with explicit prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, [my_tool], prompt)
executor = AgentExecutor(agent=agent, tools=[my_tool], verbose=True)

# 5. Invoke (not run)
result = executor.invoke({"input": "Do something"})
print(result["output"])
```

### Migration Cheat Sheet

| Deprecated | Modern Replacement |
|---|---|
| `from langchain.llms import OpenAI` | `from langchain_openai import ChatOpenAI` |
| `LLMChain(llm=..., prompt=...)` | `prompt \| llm` (LCEL) or agent |
| `Tool.from_function(func=..., name=..., description=...)` | `@tool` decorator |
| `initialize_agent(tools, llm)` | `create_tool_calling_agent(llm, tools, prompt)` + `AgentExecutor(...)` |
| `agent.run("query")` | `executor.invoke({"input": "query"})` |
| `chain("input")` | `chain.invoke("input")` |

In [ ]:
# Live demo: Modern pattern in action
# This is the clean, modern way to build an agent

@tool
def reverse_string(text: str) -> str:
    """Reverse the given string. Returns the string with characters in reverse order."""
    return text[::-1]

@tool
def uppercase_string(text: str) -> str:
    """Convert the given string to uppercase. Returns the string with all characters capitalized."""
    return text.upper()

# Modern pattern: explicit prompt, create_tool_calling_agent, AgentExecutor
modern_tools = [reverse_string, uppercase_string]

modern_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a string manipulation assistant. Use your tools to transform text as requested."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

modern_agent = create_tool_calling_agent(llm, modern_tools, modern_prompt)
modern_executor = AgentExecutor(
    agent=modern_agent,
    tools=modern_tools,
    verbose=True
)

result = modern_executor.invoke(
    {"input": "Reverse the word 'hello' and then convert the result to uppercase"}
)
print("\nFinal Answer:", result["output"])

---

## Exercise 3: Error Handling and Retry Logic

Build an agent with a tool that randomly fails, and implement proper error handling with retry logic.

**Tasks:**
1. Create an `unreliable_api` tool that fails ~50% of the time
2. Create a `safe_api_call` wrapper tool that adds retry logic (up to 3 attempts)
3. Build an agent using both tools and test it

In [ ]:
# Exercise 3: YOUR CODE HERE

# Step 1: Create an unreliable tool
@tool
def unreliable_api(query: str) -> str:
    """Query an unreliable API that sometimes fails. Returns information about
    the query topic when successful."""
    # TODO: Use random.random() to simulate ~50% failure rate
    # On failure: raise an exception or return an error message
    # On success: return a mock response based on the query
    pass


# Step 2: Create a reliable wrapper with retry logic
@tool
def safe_api_call(query: str) -> str:
    """Safely call the unreliable API with automatic retry logic. Tries up to
    3 times before returning an error."""
    # TODO: Implement retry logic that calls unreliable_api up to 3 times
    # Return the result on success, or an error message after all retries fail
    pass


# Step 3: Build and test the agent
retry_tools = None  # Replace with list of tools
retry_agent = None   # Replace with agent
retry_executor = None  # Replace with AgentExecutor

# result = retry_executor.invoke({"input": "Look up information about quantum computing"})
# print(result["output"])

### Solution

In [ ]:
# Exercise 3: SOLUTION

# Step 1: Create an unreliable tool (simulates ~50% failure rate)
call_count = {"total": 0, "failures": 0}

@tool
def unreliable_api(query: str) -> str:
    """Query an unreliable API that sometimes fails. Returns information
    about the query topic when successful."""
    call_count["total"] += 1
    # Simulate 50% failure rate
    if random.random() < 0.5:
        call_count["failures"] += 1
        raise ConnectionError(f"API connection failed (attempt #{call_count['total']})")

    # Mock successful responses
    responses = {
        "quantum": "Quantum computing uses qubits that can exist in superposition, enabling parallel computation. Key players include IBM, Google, and IonQ.",
        "ai": "Artificial Intelligence has seen rapid advances with transformer models, achieving human-level performance on many benchmarks.",
        "climate": "Global temperatures have risen 1.1C since pre-industrial times. Renewable energy adoption is accelerating worldwide.",
    }
    query_lower = query.lower()
    for keyword, response in responses.items():
        if keyword in query_lower:
            return response
    return f"Retrieved general information about: {query}"


# Step 2: Create a reliable wrapper with retry logic
@tool
def safe_api_call(query: str) -> str:
    """Safely query information with automatic retry logic. Retries up to 3
    times if the underlying API fails. Use this instead of unreliable_api
    for reliable results."""
    max_retries = 3
    errors = []
    for attempt in range(1, max_retries + 1):
        try:
            # Directly call the underlying logic (not the tool object)
            call_count["total"] += 1
            if random.random() < 0.5:
                call_count["failures"] += 1
                raise ConnectionError(f"Connection failed on attempt {attempt}")

            # Mock responses
            responses = {
                "quantum": "Quantum computing uses qubits that can exist in superposition, enabling parallel computation. Key players include IBM, Google, and IonQ.",
                "ai": "Artificial Intelligence has seen rapid advances with transformer models, achieving human-level performance on many benchmarks.",
                "climate": "Global temperatures have risen 1.1C since pre-industrial times. Renewable energy adoption is accelerating worldwide.",
            }
            query_lower = query.lower()
            for keyword, response in responses.items():
                if keyword in query_lower:
                    return f"[Success on attempt {attempt}] {response}"
            return f"[Success on attempt {attempt}] General information about: {query}"

        except Exception as e:
            errors.append(f"Attempt {attempt}: {str(e)}")
            if attempt < max_retries:
                continue

    return f"All {max_retries} attempts failed. Errors: {'; '.join(errors)}"


# Step 3: Build the agent
retry_tools = [safe_api_call, calculator_tool]

retry_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful research assistant. Use safe_api_call to look up information. "
        "If a lookup fails after retries, inform the user that the information is temporarily unavailable."
    )),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

retry_agent = create_tool_calling_agent(llm, retry_tools, retry_prompt)
retry_executor = AgentExecutor(
    agent=retry_agent,
    tools=retry_tools,
    verbose=True,
    handle_parsing_errors=True
)

# Reset counters and test
call_count = {"total": 0, "failures": 0}

result = retry_executor.invoke(
    {"input": "Look up information about quantum computing"}
)
print("\nFinal Answer:", result["output"])
print(f"\nAPI Stats: {call_count['total']} total calls, {call_count['failures']} failures")

---

## 8. Summary and Key Takeaways

### What We Covered

1. **The Agent Loop**: Agents work in an Observe-Think-Act cycle, repeatedly calling tools until they have enough information to answer.

2. **Building Tools**: The `@tool` decorator is the modern way to create tools in LangChain. It auto-generates schemas from type hints and docstrings.

3. **Agent Types**:
   - **Tool-Calling Agent**: Uses native LLM function calling. Best for production with modern models.
   - **ReAct Agent**: Generates explicit Thought/Action/Observation traces. Best for debugging and transparency.

4. **Error Handling**: Production agents need robust error handling at the tool level (try/except), retry logic, and graceful degradation.

5. **API Migration**: Modern LangChain uses `@tool`, `create_tool_calling_agent`, and `AgentExecutor` instead of the deprecated `Tool.from_function`, `LLMChain`, and `initialize_agent`.

### Coming Up Next

**Module 13: Multi-Agent Systems** will cover:
- Orchestrating multiple specialized agents
- Agent-to-agent communication
- Supervisor patterns and hierarchical agent architectures
- Building on the sequential chain pattern from `ai_trading_agent.ipynb` with modern APIs

### References

- **Paper**: Yao, S., et al. (2023). ["ReAct: Synergizing Reasoning and Acting in Language Models"](https://arxiv.org/abs/2210.03629). ICLR 2023.
- **Documentation**: [LangChain Agents](https://python.langchain.com/docs/how_to/#agents) (latest)
- **Course**: DeepLearning.AI - ["Functions, Tools and Agents with LangChain"](https://www.deeplearning.ai/short-courses/functions-tools-agents-langchain/)
- **Paper**: Schick, T., et al. (2023). ["Toolformer: Language Models Can Teach Themselves to Use Tools"](https://arxiv.org/abs/2302.04761).
- **Paper**: Wei, J., et al. (2022). ["Chain-of-Thought Prompting Elicits Reasoning in Large Language Models"](https://arxiv.org/abs/2201.11903). NeurIPS 2022.